In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

def generate_synthetic_vitals(num_patients: int = 100, hours: int = 24) -> pd.DataFrame:
    """
    Generates realistic vitals with:
    - SpO₂/SBP as integers
    - RR ≥30 for disease states
    - Controlled missing data (5% RR only)
    - Includes age column (1 to 90 years)
    """
    np.random.seed(42)
    data = []

    for pid in range(num_patients):
        base_time = datetime.now() - timedelta(hours=hours)
        timestamps = [base_time + timedelta(hours=h) for h in range(hours)]

        # Assign each patient a fixed age between 1 and 90
        age = np.random.randint(1, 91)

        pattern = np.random.choice(
            ['normal', 'pneumonia', 'copd', 'asthma'],
            p=[0.5, 0.2, 0.2, 0.1]
        )

        for t in timestamps:
            if pattern == 'normal':
                spo2 = int(np.random.normal(97, 1.5))
                rr = np.random.randint(12, 20)
                hr = np.random.randint(60, 100)
                temp = round(np.random.normal(36.8, 0.3), 1)
                sbp = int(np.random.normal(120, 10))

            elif pattern == 'pneumonia':
                spo2 = int(np.random.normal(92, 3))
                rr = np.random.randint(22, 35)
                hr = np.random.randint(90, 120)
                temp = round(np.random.normal(38.5, 0.5), 1)
                sbp = int(np.random.normal(110, 15))

            elif pattern == 'copd':
                spo2 = int(np.random.normal(90, 2))
                rr = np.random.randint(20, 30)
                hr = np.random.randint(80, 110)
                temp = round(np.random.normal(37.0, 0.2), 1)
                sbp = int(np.random.normal(130, 12))

            elif pattern == 'asthma':
                rr_choice = np.random.choice([np.random.randint(12, 18), np.random.randint(25, 35)], p=[0.7, 0.3])
                spo2 = int(np.random.normal(94, 2))
                rr = rr_choice
                hr = np.random.randint(70, 110)
                temp = round(np.random.normal(36.9, 0.3), 1)
                sbp = int(np.random.normal(125, 10))

            # Force SpO₂/SBP to valid ranges
            spo2 = max(70, min(100, spo2))
            sbp = max(80, min(200, sbp))

            # Add missing RR (5% of rows)
            if np.random.rand() < 0.05:
                rr = np.nan

            # Add outliers (1% chance)
            if np.random.rand() < 0.01:
                spo2 = np.random.choice([45, 101])

            data.append([pid, age, t, spo2, rr, hr, temp, sbp])

    df = pd.DataFrame(data, columns=['patient_id', 'age', 'timestamp', 'spo2', 'resp_rate', 'heart_rate', 'temperature', 'sbp'])
    return df


In [2]:
df = generate_synthetic_vitals()
df.to_csv('synthetic_data8.csv', index=False)
display(df.head())

,patient_id,age,timestamp,spo2,resp_rate,heart_rate,temperature,sbp
0,0,52,2025-11-07 16:21:03.710558,93,14.0,80,36.8,104
1,0,52,2025-11-07 17:21:03.710558,93,26.0,99,36.7,119
2,0,52,2025-11-07 18:21:03.710558,91,17.0,97,37.0,127
3,0,52,2025-11-07 19:21:03.710558,92,27.0,76,36.7,134
4,0,52,2025-11-07 20:21:03.710558,93,15.0,86,37.0,133


In [3]:
def assign_age_category(df):
    df['age_category'] = df['age'].apply(
        lambda a: 'infant' if a < 1 else 'child' if a < 18 else 'adult'
    )
    return df


In [4]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
# Thresholds
THRESH_SPO2_LOW = 92
THRESH_SPO2_CRITICAL = 88
THRESH_SHOCK_INDEX_HIGH = 0.9
THRESH_TEMP_HIGH = 38.0

# Trend Window
TREND_WINDOW = 6

AGE_THRESHOLDS = {
    'infant': {
        'rr_low': 30,
        'rr_normal': 40,
        'rr_high': 60,
        'hr_low': 80,
        'hr_normal': 120,
        'hr_high': 160,
        'sbp_low': 70,
        'sbp_normal': 90,
        'sbp_high': 110,
        'temp_low': 35.5,
        'temp_normal': 36.8,
        'temp_high': 38.0
    },
    'child': {
        'rr_low': 14,
        'rr_normal': 20,
        'rr_high': 30,
        'hr_low': 70,
        'hr_normal': 90,
        'hr_high': 110,
        'sbp_low': 90,
        'sbp_normal': 105,
        'sbp_high': 120,
        'temp_low': 35.5,
        'temp_normal': 36.8,
        'temp_high': 38.0
    },
    'adult': {
        'rr_low': 12,
        'rr_normal': 16,
        'rr_high': 20,
        'hr_low': 60,
        'hr_normal': 80,
        'hr_high': 100,
        'sbp_low': 100,
        'sbp_normal': 115,
        'sbp_high': 130,
        'temp_low': 35.5,
        'temp_normal': 36.8,
        'temp_high': 38.0
    }
}




# Score Thresholds
SCORE_THRESHOLDS = {
    "pneumonia": 6,
    "copd": 6,
    "asthma": 5
}


In [5]:
def apply_vital_range_flags(df):
    df = df.copy()
    df = assign_age_category(df)  # Inject age group

    # SpO₂ flags
    df['flag_spo2_low'] = df['spo2'] < THRESH_SPO2_LOW

    # Temperature flags (both high & low for hypothermia detection)
    df['flag_temp_high'] = df['temperature'] >= AGE_THRESHOLDS[df['age_category'].iloc[0]]['temp_high']
    df['flag_temp_low'] = df['temperature'] < AGE_THRESHOLDS[df['age_category'].iloc[0]]['temp_low']

    # RR flags
    df['flag_rr_high'] = df.apply(lambda row: row['resp_rate'] >= AGE_THRESHOLDS[row['age_category']]['rr_high'], axis=1)

    # HR flags
    df['flag_hr_high'] = df.apply(lambda row: row['heart_rate'] >= AGE_THRESHOLDS[row['age_category']]['hr_high'], axis=1)

    # SBP flags
    df['flag_sbp_low'] = df.apply(lambda row: row['sbp'] < AGE_THRESHOLDS[row['age_category']]['sbp_low'], axis=1)

    return df


In [6]:


def compute_recent_trends_delta(df):
    """
    Computes trends for each vital by differencing consecutive readings.
    Applies stricter interpretation using age-specific thresholds.
    """
    df = df.copy().sort_values("timestamp").reset_index(drop=True)

    if 'age_category' not in df.columns:
        df = assign_age_category(df)

    trends = {}
    recent = df.tail(TREND_WINDOW)
    age_group = recent['age_category'].iloc[-1]
    thresholds = AGE_THRESHOLDS[age_group]

    vital_map = {
        'resp_rate': ('rr_low', 'rr_normal', 'rr_high'),
        'heart_rate': ('hr_low', 'hr_normal', 'hr_high'),
        'sbp': ('sbp_low', 'sbp_normal', 'sbp_high'),
        'temperature': ('temp_low', 'temp_normal', 'temp_high'),
        'spo2': (None, None, None)  # handled separately
    }

    for vital in ['resp_rate', 'heart_rate', 'sbp', 'temperature', 'spo2']:
        if vital not in recent.columns or recent[vital].isnull().all():
            continue

        y = recent[vital].dropna().values
        if len(y) < 2:
            continue

        avg_delta = np.mean(np.diff(y))
        latest = y[-1]
        trends[f"{vital}_trend"] = round(avg_delta, 3)

        if vital == 'spo2':
            if latest < THRESH_SPO2_LOW:
                if avg_delta > 0:
                    flag = "Still abnormal — but improving"
                elif avg_delta < 0:
                    flag = "Abnormal and worsening"
                else:
                    flag = "Abnormal and flat"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        else:
            low_key, norm_key, high_key = vital_map[vital]
            low = thresholds[low_key]
            normal = thresholds[norm_key]
            high = thresholds[high_key]

            if latest < low or latest > high:
                if (latest > high and avg_delta < 0) or (latest < low and avg_delta > 0):
                    flag = "Still abnormal — but improving"
                else:
                    flag = "Abnormal and worsening"
            else:
                if avg_delta < 0:
                    flag = "Normal but deteriorating"
                else:
                    flag = "Normal and stable"

        trends[f"{vital}_trend_flag"] = flag

    # Shock Index trend
    if all(col in recent.columns for col in ['heart_rate', 'sbp']):
        hr = recent['heart_rate'].values
        sbp = np.clip(recent['sbp'].values, a_min=1, a_max=None)
        si = hr / sbp

        if len(si) >= 2:
            avg_si_delta = np.mean(np.diff(si))
            trends['shock_index_trend'] = round(avg_si_delta, 3)

            latest_si = si[-1]
            if latest_si >= THRESH_SHOCK_INDEX_HIGH:
                flag = "Shock Index high — improving" if avg_si_delta < 0 else "Shock Index high — worsening"
            else:
                flag = "Normal but improving" if avg_si_delta < 0 else "Normal but rising"

            trends['shock_index_trend_flag'] = flag

    return trends


In [7]:
def apply_combination_patterns(df):
    df = df.copy()

    # Fill missing vitals with safe defaults
    df['heart_rate'] = df['heart_rate'].fillna(0)
    df['sbp'] = df['sbp'].clip(lower=1).fillna(120)
    df['spo2'] = df['spo2'].fillna(100)
    df['temperature'] = df['temperature'].fillna(36.5)
    df['resp_rate'] = df['resp_rate'].fillna(18)

    # Ensure age category is defined
    if 'age_category' not in df.columns:
        df = assign_age_category(df)

    # Compute Shock Index
    df['shock_index'] = df['heart_rate'] / df['sbp']

    # Initialize infection pattern column
    df['pattern_infection'] = False

    # INFANT infection pattern (broader criteria)
    infant_infection_criteria = (
        (df['flag_temp_high'] | df['flag_temp_low']) |
        df['flag_hr_high'] |
        df['flag_rr_high']
    )
    df.loc[df['age_category'] == 'infant', 'pattern_infection'] = infant_infection_criteria

    #  NON-INFANT infection pattern (standard strict logic)
    non_infant_infection_criteria = df['flag_temp_high'] & df['flag_spo2_low']
    df.loc[df['age_category'] != 'infant', 'pattern_infection'] = non_infant_infection_criteria

    # OBSTRUCTIVE pattern (for all ages)
    df['pattern_obstructive'] = (
        df['flag_rr_high'] & (df['spo2'] < 94) & df['flag_hr_high']
    )

    #  SHOCK pattern
    df['pattern_shock'] = df['shock_index'] >= 0.9

    #  MIXED pattern: ≥3 abnormal vital signs
    df['pattern_mixed'] = (
        df[['flag_spo2_low', 'flag_rr_high', 'flag_hr_high', 'flag_temp_high', 'flag_temp_low', 'flag_sbp_low']].sum(axis=1) >= 3
    )

    # Mixed Infectious + Obstructive pattern
    df['pattern_mixed_infectious_obstructive'] = (
        df['pattern_infection'] & df['pattern_obstructive']
    )

    return df



In [8]:
def check_safety_overrides(df):
    """
    Applies safety escalation rules on a patient's vitals DataFrame.
    Assumes 'timestamp', 'spo2', 'heart_rate', 'sbp', 'resp_rate', 'temperature', 'age_category' columns exist.
    Returns a list of override alerts.
    """
    overrides = []
    df = df.sort_values("timestamp").reset_index(drop=True)

    # Ensure necessary columns exist
    required_cols = ['spo2', 'heart_rate', 'sbp', 'resp_rate', 'temperature', 'age_category']
    if not all(col in df.columns for col in required_cols):
        return ["[CRITICAL] Missing vital data for safety checks"]

    df['shock_index'] = df['heart_rate'] / df['sbp'].clip(lower=1)

    # Rule 1: SpO₂ ≤ 88% sustained for ≥5 minutes (i.e., ≥5 consecutive readings if 1 reading per minute)
    low_spo2_streak = (df['spo2'] <= 88).astype(int).rolling(window=5, min_periods=1).sum()
    if (low_spo2_streak >= 5).any():
        overrides.append("[CRITICAL] Sustained SpO₂ ≤ 88% for ≥5 mins — Immediate clinician review")

    # Rule 2: ShockIndex ≥ 0.9 with temp burden ≥ 30%
    temp_burden = (df['temperature'] >= 38.0).mean()
    high_si = (df['shock_index'] >= 0.9).any()
    if high_si and temp_burden >= 0.3:
        overrides.append("[CRITICAL] Shock Index high + prolonged fever — Possible sepsis — Activate screen")

    # Rule 3: SBP < 90 for ≥2 hours (assuming 1 reading/min → 120 readings)
    sbp_low_streak = (df['sbp'] < 90).astype(int).rolling(window=120, min_periods=1).sum()
    if (sbp_low_streak >= 120).any():
        overrides.append("[CRITICAL] SBP < 90 mmHg for ≥2 hours — Urgent escalation")

    # Rule 4: RR ≥ 30 for ≥2 hours + SpO₂_median ≤ 90%
    rr_high_streak = (df['resp_rate'] >= 30).astype(int).rolling(window=120, min_periods=1).sum()
    if (rr_high_streak >= 120).any() and df['spo2'].median() <= 90:
        overrides.append("[CRITICAL] RR ≥ 30 + low SpO₂ — Risk of respiratory failure")

    # Rule 5: Infant-specific override: Temp high/low + HR/RR high
    if df['age_category'].iloc[-1] == 'infant':
        last_temp = df['temperature'].iloc[-1]
        last_hr = df['heart_rate'].iloc[-1]
        last_rr = df['resp_rate'].iloc[-1]
        thresholds = AGE_THRESHOLDS['infant']

        if (last_temp >= thresholds['temp_high'] or last_temp <= thresholds['temp_low']) and \
           (last_hr >= thresholds['hr_high']) and \
           (last_rr >= thresholds['rr_high']):
            overrides.append("[CRITICAL] Signs of infection in infant — fever or hypothermia + fast HR + fast RR")

    if not overrides:
        overrides.append("[LOW] No safety overrides triggered")

    return overrides


In [9]:

from scipy.stats import linregress

def compute_rmssd(arr):
    """Compute RMSSD (Root Mean Square of Successive Differences)"""
    diffs = np.diff(arr.dropna())
    return np.sqrt(np.mean(diffs**2)) if len(diffs) > 1 else 0

def compute_derived_features(df_12h, df_24h):
    features = {}

    # Median SpO2
    features['SPO2_median_12'] = df_12h['spo2'].median()
    features['SPO2_median_24'] = df_24h['spo2'].median()

    # SpO2 Trend (points/hour)
    if len(df_12h) > 1:
        x_hours = (df_12h['timestamp'] - df_12h['timestamp'].iloc[0]).dt.total_seconds() / 3600
        slope_spo2, _, _, _, _ = linregress(x_hours, df_12h['spo2'])
        features['SPO2_trend_12'] = round(slope_spo2, 3)
    else:
        features['SPO2_trend_12'] = 0

    # SpO2 Burden
    features['SPO2_burden92_12'] = (df_12h['spo2'] < 92).mean()
    features['SPO2_burden92_24'] = (df_24h['spo2'] < 92).mean()

    # RR Burden
    features['RR_burden22_12'] = (df_12h['resp_rate'] >= 22).mean()
    features['RR_burden22_24'] = (df_24h['resp_rate'] >= 22).mean()

    # HR Burden
    features['HR_burden100_12'] = (df_12h['heart_rate'] >= 100).mean()
    features['HR_burden100_24'] = (df_24h['heart_rate'] >= 100).mean()

    # Temp Burden
    features['Temp_burden38_12'] = (df_12h['temperature'] >= 38.0).mean()
    features['Temp_burden38_24'] = (df_24h['temperature'] >= 38.0).mean()

    # SBP Trend
    if len(df_12h) > 1:
        x_hours = (df_12h['timestamp'] - df_12h['timestamp'].iloc[0]).dt.total_seconds() / 3600
        slope_sbp, _, _, _, _ = linregress(x_hours, df_12h['sbp'])
        features['SBP_trend_12'] = round(slope_sbp, 3)
    else:
        features['SBP_trend_12'] = 0

    if len(df_24h) > 1:
        x_hours_24 = (df_24h['timestamp'] - df_24h['timestamp'].iloc[0]).dt.total_seconds() / 3600
        slope_sbp_24, _, _, _, _ = linregress(x_hours_24, df_24h['sbp'])
        features['SBP_trend_24'] = round(slope_sbp_24, 3)
    else:
        features['SBP_trend_24'] = 0

    # RMSSD
    features['RMSSD_spo2_12'] = compute_rmssd(df_12h['spo2'])
    features['RMSSD_rr_12'] = compute_rmssd(df_12h['resp_rate'])

    # Shock Index Median (24h)
    si = df_24h['heart_rate'] / df_24h['sbp'].clip(lower=1)
    features['ShockIndex_med'] = si.median()

    # Nocturnal drop (SpO2)
    df_24h['hour'] = df_24h['timestamp'].dt.hour
    night_spo2 = df_24h[df_24h['hour'].between(0, 6)]['spo2']
    day_spo2 = df_24h[df_24h['hour'].between(8, 20)]['spo2']
    features['Nocturnal_drop'] = night_spo2.median() - day_spo2.median()

    return features


In [10]:
def score_pneumonia_theory(df):
    score_12 = 0
    score_24 = 0

    # 12h window
    if df['Temp_burden38_12'] >= 0.3: score_12 += 3
    if df['SPO2_trend_12'] <= -0.15: score_12 += 2
    if df['SPO2_burden92_12'] >= 0.2: score_12 += 2
    if df['RR_burden22_12'] >= 0.3: score_12 += 1
    if df['HR_burden100_12'] >= 0.3: score_12 += 1
    if df['SBP_trend_12'] <= -1: score_12 += 1

    # 24h window
    if df['Temp_burden38_24'] >= 0.3: score_24 += 3
    if df['SPO2_burden92_24'] >= 0.2: score_24 += 2
    if df['RR_burden22_24'] >= 0.3: score_24 += 1
    if df['HR_burden100_24'] >= 0.3: score_24 += 1
    if df['SBP_trend_24'] <= -1: score_24 += 1

    return score_12, score_24


In [11]:
def score_copd_theory(df):
    score_12 = 0
    score_24 = 0

    # 12h window
    if df['Temp_burden38_12'] <= 0.1: score_12 += 2
    if df['SPO2_median_12'] <= 92: score_12 += 3
    if df['SPO2_burden92_12'] >= 0.4: score_12 += 2
    if df['RR_burden22_12'] >= 0.3: score_12 += 1
    if 0.1 <= df['HR_burden100_12'] <= 0.4: score_12 += 1
    # Optional nocturnal drop (if available)
    if 'Nocturnal_drop' in df and df['Nocturnal_drop'] <= -2: score_12 += 1

    # 24h window
    if df['Temp_burden38_24'] <= 0.1: score_24 += 2
    if df['SPO2_median_24'] <= 92: score_24 += 3
    if df['SPO2_burden92_24'] >= 0.4: score_24 += 2
    if df['RR_burden22_24'] >= 0.3: score_24 += 1
    if 0.1 <= df['HR_burden100_24'] <= 0.4: score_24 += 1

    return score_12, score_24


In [12]:
def score_asthma_theory(df):
    score_12 = 0
    score_24 = 0

    # 12h window
    if df['Temp_burden38_12'] <= 0.1: score_12 += 1
    if df['SPO2_median_12'] >= 94 and 0.05 <= df['SPO2_burden92_12'] <= 0.2: score_12 += 2
    if df['RMSSD_spo2_12'] >= 1.5 or df['RMSSD_rr_12'] >= 4: score_12 += 2
    if df['RR_burden22_12'] >= 0.3 and df['SPO2_trend_12'] >= -0.05: score_12 += 2
    if 0.1 <= df['HR_burden100_12'] <= 0.4: score_12 += 1

    # 24h window
    if df['Temp_burden38_24'] <= 0.1: score_24 += 1
    if df['SPO2_median_24'] >= 94 and 0.05 <= df['SPO2_burden92_24'] <= 0.2: score_24 += 2
    if df['RMSSD_spo2_12'] >= 1.5 or df['RMSSD_rr_12'] >= 4: score_24 += 2
    if df['RR_burden22_24'] >= 0.3 and df['SPO2_trend_12'] >= -0.05: score_24 += 2
    if 0.1 <= df['HR_burden100_24'] <= 0.4: score_24 += 1

    return score_12, score_24


In [13]:
def detect_disease_from_features(features):
    diagnosis = []

    # Pneumonia logic
    pneumonia_score = 0
    if features.get('Temp_burden38_12', 0) >= 0.3: pneumonia_score += 3
    if features.get('SPO2_trend_12', 0) <= -0.15: pneumonia_score += 2
    if features.get('SPO2_burden92_24', 0) >= 0.2: pneumonia_score += 2
    if features.get('RR_burden22_24', 0) >= 0.3: pneumonia_score += 1
    if features.get('HR_burden100_24', 0) >= 0.3: pneumonia_score += 1
    if features.get('SBP_trend_12', 0) <= -1: pneumonia_score += 1

    # COPD logic
    copd_score = 0
    if features.get('Temp_burden38_12', 0) <= 0.1: copd_score += 2
    if features.get('SPO2_median_24', 100) <= 92: copd_score += 3
    if features.get('SPO2_burden92_24', 0) >= 0.4: copd_score += 2
    if features.get('RR_burden22_24', 0) >= 0.3: copd_score += 1
    if features.get('Nocturnal_drop', 0) <= -2: copd_score += 1
    if 0.1 <= features.get('HR_burden100_24', 0) <= 0.4: copd_score += 1

    # Asthma logic
    asthma_score = 0
    if features.get('Temp_burden38_12', 0) <= 0.1: asthma_score += 1
    if features.get('SPO2_median_24', 0) >= 94 and 0.05 <= features.get('SPO2_burden92_24', 0) <= 0.2:
        asthma_score += 2
    if features.get('RMSSD_spo2_12', 0) >= 1.5 or features.get('RMSSD_rr_12', 0) >= 4:
        asthma_score += 2
    if features.get('RR_burden22_24', 0) >= 2 and features.get('SPO2_trend_12', 0) >= -0.05:
        asthma_score += 2
    if 0.1 <= features.get('HR_burden100_24', 0) <= 0.4: asthma_score += 1

    # Print each score
    print(f"Pneumonia Score: {pneumonia_score}")
    print(f"COPD Score: {copd_score}")
    print(f"Asthma Score: {asthma_score}")

    # Apply final thresholds
    if pneumonia_score >= 6 and copd_score < 6 and asthma_score < 5:
        return "Pneumonia-suggestive"
    elif copd_score >= 6 and pneumonia_score < 6 and asthma_score < 5:
        return "COPD-exacerbation-suggestive"
    elif asthma_score >= 5 and pneumonia_score < 6 and copd_score < 6:
        return "Asthma-exacerbation-suggestive"
    elif pneumonia_score >= 6 and copd_score >= 6:
        return "Mixed obstructive + infectious"
    else:
        return "No clear disease suggestion"


In [ ]:
import pandas as pd

# Load CSV
df = pd.read_csv("synthetic_data9.csv")

# Ensure timestamp is in datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'])


In [23]:
def analyze_patient(df_patient):
    # Run the steps in order
    df_patient = assign_age_category(df_patient)
    df_patient = apply_vital_range_flags(df_patient)
    df_patient = apply_combination_patterns(df_patient)
    safety_overrides = check_safety_overrides(df_patient)
    trend_flags = compute_recent_trends_delta(df_patient)

    # Time-based subsets
    now = df_patient['timestamp'].max()
    df_12h = df_patient[df_patient['timestamp'] >= now - pd.Timedelta(hours=12)].copy()
    df_24h = df_patient[df_patient['timestamp'] >= now - pd.Timedelta(hours=24)].copy()

    # Assign age category for subsets
    df_12h = assign_age_category(df_12h)
    df_24h = assign_age_category(df_24h)

    # Derived features
    features = compute_derived_features(df_12h, df_24h)

    # Scores
    pn_score_12, pn_score_24 = score_pneumonia_theory(features)
    copd_score_12, copd_score_24 = score_copd_theory(features)
    asthma_score_12, asthma_score_24 = score_asthma_theory(features)

    # Diagnosis
    if pn_score_12 >= 6 and pn_score_24 >= 6:
        diagnosis = "Pneumonia-suggestive"
        prompts = ["Order chest X-ray", "Start pneumonia pathway"]
    elif copd_score_12 >= 6 and copd_score_24 >= 6:
        diagnosis = "COPD-exacerbation-suggestive"
        prompts = ["Start COPD management", "Assess oxygen therapy"]
    elif asthma_score_12 >= 5 and asthma_score_24 >= 5:
        diagnosis = "Asthma-exacerbation-suggestive"
        prompts = ["Start asthma protocol", "Administer bronchodilator"]
    elif pn_score_24 >= 6 and copd_score_24 >= 6:
        diagnosis = "Mixed obstructive + infectious"
        prompts = ["Check for co-infection", "Dual-pathway treatment"]
    else:
        diagnosis = "No clear disease suggestion"
        prompts = []

    # Safety flags
    safety_flags = {
        "shock_index_med": round(features.get('ShockIndex_med', 0), 2),
        "spo2_crit": features.get('SPO2_burden92_24', 0) >= 0.5
    }

    # Features to display
    selected_features = {
        "spo2_median_24": round(features.get("SPO2_median_24", 0), 2),
        "spo2_trend_12": round(features.get("SPO2_trend_12", 0), 2),
        "spo2_burden92_24": round(features.get("SPO2_burden92_24", 0), 2),
        "rr_burden22_24": round(features.get("RR_burden22_24", 0), 2),
        "hr_burden100_24": round(features.get("HR_burden100_24", 0), 2),
        "temp_burden38_24": round(features.get("Temp_burden38_24", 0), 2)
    }

    # Output (human-readable)
    print(f" Patient ID: {df_patient['patient_id'].iloc[0]}")
    print(f"Diagnosis: {diagnosis}")
    print(f"Scores: Pneumonia(12h/24h) = {pn_score_12}/{pn_score_24}, COPD(12h/24h) = {copd_score_12}/{copd_score_24}, Asthma(12h/24h) = {asthma_score_12}/{asthma_score_24}")
    print("Selected Features:")
    for k, v in selected_features.items():
        print(f"  {k}: {v}")
    print("Trend Flags:", trend_flags)
    print("Safety Flags:", safety_flags)
    print("Prompts:", prompts)
    print("Safety Overrides:", safety_overrides)
    print("\n")


In [ ]:
for pid, df_patient in df.groupby("patient_id"):
    analyze_patient(df_patient.sort_values("timestamp"))


--- Patient ID: 0 ---
Diagnosis: Asthma-exacerbation-suggestive
Scores: Pneumonia(12h/24h) = 1/1, COPD(12h/24h) = 5/4, Asthma(12h/24h) = 8/8
Selected Features:
  spo2_median_24: 94.0
  spo2_trend_12: 0.47
  spo2_burden92_24: 0.1
  rr_burden22_24: 1.0
  hr_burden100_24: 0.2
  temp_burden38_24: 0.0
Trend Flags: {'resp_rate_trend': np.float64(-0.2), 'resp_rate_trend_flag': 'Still abnormal — but improving', 'heart_rate_trend': np.float64(1.8), 'heart_rate_trend_flag': 'Normal and stable', 'sbp_trend': np.float64(0.2), 'sbp_trend_flag': 'Normal and stable', 'temperature_trend': np.float64(-0.22), 'temperature_trend_flag': 'Normal but deteriorating', 'spo2_trend': np.float64(0.4), 'spo2_trend_flag': 'Normal and stable', 'shock_index_trend': np.float64(0.014), 'shock_index_trend_flag': 'Normal but rising'}
Safety Flags: {'shock_index_med': np.float64(0.68), 'spo2_crit': np.False_}
Prompts: ['Start asthma protocol', 'Administer bronchodilator']
Safety Overrides: ['[LOW] No safety overrides tri

In [ ]:
df = assign_age_category(df)
df = apply_vital_range_flags(df)
df = apply_combination_patterns(df)
safety_overrides = check_safety_overrides(df)
trend_flags = compute_recent_trends_delta(df)

# Time-based subsets
now = df['timestamp'].max()
df_12h = df[df['timestamp'] >= now - timedelta(hours=12)].copy()
df_24h = df[df['timestamp'] >= now - timedelta(hours=24)].copy()

# Age category for subsets
df_12h = assign_age_category(df_12h)
df_24h = assign_age_category(df_24h)

# Features
features = compute_derived_features(df_12h, df_24h)

# Scores
pn_score_12, pn_score_24 = score_pneumonia_theory(features)
copd_score_12, copd_score_24 = score_copd_theory(features)
asthma_score_12, asthma_score_24 = score_asthma_theory(features)




In [25]:
import pandas as pd
from datetime import datetime, timedelta
timestamps = [datetime.now() - timedelta(minutes=10*i) for i in reversed(range(6))]

sample_df = pd.DataFrame({
    "patient_id": [1, 1, 1, 1, 1, 1],
    "timestamp": timestamps,
    "spo2": [95, 94, 93, 92, 91, 90],
    "temperature": [37.5, 37.7, 38.0, 38.1, 38.2, 38.3],
    "resp_rate": [4,6,5,10,8,9],
    "heart_rate": [80, 82, 85, 88, 90, 94],
    "sbp": [110, 108, 107, 106, 104, 102],
    "age": [10]*6  
})


In [26]:
sample_df= assign_age_category(sample_df)

In [27]:
sample_df['timestamp'] = pd.to_datetime(sample_df['timestamp'])


In [28]:
overrides = check_safety_overrides(sample_df)
for o in overrides:
    print("-", o)


- [CRITICAL] Shock Index high + prolonged fever — Possible sepsis — Activate screen


In [29]:
trends = compute_recent_trends_delta(sample_df)
print(trends)

{'resp_rate_trend': np.float64(1.0), 'resp_rate_trend_flag': 'Still abnormal — but improving', 'heart_rate_trend': np.float64(2.8), 'heart_rate_trend_flag': 'Normal and stable', 'sbp_trend': np.float64(-1.6), 'sbp_trend_flag': 'Normal but deteriorating', 'temperature_trend': np.float64(0.16), 'temperature_trend_flag': 'Abnormal and worsening', 'spo2_trend': np.float64(-1.0), 'spo2_trend_flag': 'Abnormal and worsening', 'shock_index_trend': np.float64(0.039), 'shock_index_trend_flag': 'Shock Index high — worsening'}


In [30]:
analyze_patient(sample_df)

 Patient ID: 1
Diagnosis: Pneumonia-suggestive
Scores: Pneumonia(12h/24h) = 8/6, COPD(12h/24h) = 0/0, Asthma(12h/24h) = 0/0
Selected Features:
  spo2_median_24: 92.5
  spo2_trend_12: -6.0
  spo2_burden92_24: 0.33
  rr_burden22_24: 0.0
  hr_burden100_24: 0.0
  temp_burden38_24: 0.67
Trend Flags: {'resp_rate_trend': np.float64(1.0), 'resp_rate_trend_flag': 'Still abnormal — but improving', 'heart_rate_trend': np.float64(2.8), 'heart_rate_trend_flag': 'Normal and stable', 'sbp_trend': np.float64(-1.6), 'sbp_trend_flag': 'Normal but deteriorating', 'temperature_trend': np.float64(0.16), 'temperature_trend_flag': 'Abnormal and worsening', 'spo2_trend': np.float64(-1.0), 'spo2_trend_flag': 'Abnormal and worsening', 'shock_index_trend': np.float64(0.039), 'shock_index_trend_flag': 'Shock Index high — worsening'}
Safety Flags: {'shock_index_med': np.float64(0.81), 'spo2_crit': np.False_}
Prompts: ['Order chest X-ray', 'Start pneumonia pathway']
Safety Overrides: ['[CRITICAL] Shock Index high +